# Train Autopilot models using a subset of data, and reuse its recipes on the whole dataset

Amazon SageMaker Autopilot is an automated machine learning (commonly referred to as AutoML) solution for tabular datasets. You can use SageMaker Autopilot in different ways: on autopilot (without any human input) or with human guidance, without code through SageMaker Studio or scripted using the AWS SDKs. This notebook will use the AWS SDKs to create an Autopilot job on a subset of data, and apply the recipes on a whole dataset. Compared with running Autopilot job directly on the whole dataset, you save cost and build models more efficiently.

## Setup Environment

In [55]:
import boto3
import botocore
import time
import json
import sagemaker
from sagemaker import AutoML, get_execution_role
from datetime import datetime, timezone
import pandas as pd

sess = sagemaker.session.Session()
role = sagemaker.get_execution_role()
region = boto3.session.Session().region_name

bucket = sess.default_bucket()
prefix = "autopilot-subset-0625"  # change this to your own prefix
sagemaker_client = boto3.client("sagemaker")
session = sagemaker.Session()

In [56]:
from time import gmtime, strftime, sleep

timestamp_suffix = strftime("%Y%m%d-%H-%M", gmtime())

## Prepare Data

In [ ]:
# s3 = boto3.client("s3")
# s3.download_file(
#     f"sagemaker-studio-agpwopwdg6o",
#     "bank-additional/bank-additional-full.csv",
#     "bank-data.csv",
# )

In [ ]:
# kick off an autopilot job, save this step if you already have a finished autopilot job
# s3 = boto3.client("s3")
# s3.download_file(
#     f"sagemaker-studio-agpwopwdg6o",
#     "bank-additional/bank-additional-full.csv",
#     "bank-data.csv",
# )
import zipfile

!wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip

with zipfile.ZipFile("bank+marketing.zip", "r") as zip_ref:
    zip_ref.extractall(".")

In [140]:
with zipfile.ZipFile("bank-additional.zip", "r") as zip_ref:
    zip_ref.extractall(".")

In [ ]:
# read the dataset
bank_df = pd.read_csv("./bank-additional/bank-additional-full.csv", sep=";")
pd.set_option("display.max_columns", 500)
bank_df

In [143]:
# Split the whole dataset to training and testing (80/20 split)
target = "y"
train_data = bank_df.sample(frac=0.8, random_state=200)

test_data = bank_df.drop(train_data.index)

test_data_no_target = test_data.drop(columns=[target])

# shift the target column from the last to the first
cols = list(train_data.columns)
cols = [cols[-1]] + cols[:-1]
train_data = train_data[cols]

### Upload the datasets to S3
train_val_file = "train_val_whole_data.csv"
train_data.to_csv(train_val_file, index=False, header=True)
train_val_data_s3_path = sess.upload_data(path=train_val_file, key_prefix=prefix + "/train")
print("Train-validation data uploaded to: " + train_val_data_s3_path)

test_file = "test_data_no_target.csv"
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = sess.upload_data(path=test_file, key_prefix=prefix + "/test")
print("Test data uploaded to: " + test_data_s3_path)

# further split the whole training dataset into training and validation datasets (75/25 split)
# these will be used to re-fit model using automl's recipes
val_data = train_data.sample(frac=0.25, random_state=200)
train_data = train_data.drop(val_data.index)

### Upload the datasets to S3
train_file = "train_whole_data.csv"
train_data.to_csv(train_file, index=False, header=False)
train_data_s3_path = sess.upload_data(path=train_file, key_prefix=prefix + "/train")
print("Train data uploaded to: " + train_data_s3_path)

val_file = "val_whole_data.csv"
val_data.to_csv(val_file, index=False, header=False)
val_data_s3_path = sess.upload_data(path=val_file, key_prefix=prefix + "/val")
print("validation data uploaded to: " + val_data_s3_path)

# take a subset of the training data to run autopilot job, e.g. 20%
# TODO: stratified sampling

train_subset = train_data.sample(frac=0.2, random_state=100)

# keep header for subset data, since autopilot need header information to pick target
train_file = "train_subset_data.csv"
train_subset.to_csv(train_file, index=False, header=True)
train_subset_s3_path = sess.upload_data(path=train_file, key_prefix=prefix + "/train")
print("Train subset uploaded to: " + train_subset_s3_path)

Train-validation data uploaded to: s3://sagemaker-us-east-1-827930657850/autopilot-subset-0625/train/train_val_whole_data.csv
Test data uploaded to: s3://sagemaker-us-east-1-827930657850/autopilot-subset-0625/test/test_data_no_target.csv
Train data uploaded to: s3://sagemaker-us-east-1-827930657850/autopilot-subset-0625/train/train_whole_data.csv
validation data uploaded to: s3://sagemaker-us-east-1-827930657850/autopilot-subset-0625/val/val_whole_data.csv
Train subset uploaded to: s3://sagemaker-us-east-1-827930657850/autopilot-subset-0625/train/train_subset_data.csv


## Setup Autopilot Jobs

Kick off an autopilot job using only a subset of the training data.

For baselining purpose, we also kick off an autopilot job using the whole training data. Note that the solution itself only needs the first autopilot job.

In [72]:
input_s3_path = train_subset_s3_path
auto_ml_job_name = "auto-bank-sub-" + timestamp_suffix

input_data_config = [
    {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": input_s3_path,
            }
        },
        "TargetAttributeName": target,
    }
]

job_config = {"CompletionCriteria": {"MaxCandidates": 10}}

output_data_config = {"S3OutputPath": "s3://{}/{}/full-output".format(bucket, prefix)}

print("AutoMLJobName: " + auto_ml_job_name)

sagemaker_client.create_auto_ml_job(
    AutoMLJobName=auto_ml_job_name,
    InputDataConfig=input_data_config,
    OutputDataConfig=output_data_config,
    AutoMLJobConfig=job_config,
    RoleArn=role,
)

In [72]:
# Autopilot job for baselining purpose
input_s3_path = train_val_data_s3_path
auto_ml_baseline_job_name = "auto-bank-all-" + timestamp_suffix

input_data_config = [
    {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": input_s3_path,
            }
        },
        "TargetAttributeName": target,
    }
]

job_config = {"CompletionCriteria": {"MaxCandidates": 10}}

output_data_config = {"S3OutputPath": "s3://{}/{}/full-output".format(bucket, prefix)}

print("AutoMLJobName: " + auto_ml_baseline_job_name)

sagemaker_client.create_auto_ml_job(
    AutoMLJobName=auto_ml_baseline_job_name,
    InputDataConfig=input_data_config,
    OutputDataConfig=output_data_config,
    AutoMLJobConfig=job_config,
    RoleArn=role,
)

## Tracking Autopilot Job Progress

In [ ]:
print("JobStatus - Secondary Status")
print("------------------------------")


describe_response = sagemaker_client.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print(describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"])
job_run_status = describe_response["AutoMLJobStatus"]

while job_run_status not in ("Failed", "Completed", "Stopped"):
    describe_response = sagemaker_client.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response["AutoMLJobStatus"]

    print(
        describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"]
    )
    sleep(60)

JobStatus - Secondary Status
------------------------------
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - MergingAutoMLTaskReports
InProgress - MergingAutoMLTaskReports
InProgress - MergingAutoMLTaskReports
InProgress - MergingAutoMLTaskReports
InProgress - MergingAutoMLTaskReports
InProgress - MergingAutoMLTaskReports
InProgress - MergingAutoMLTaskReports
InProgr

## Retrieve best models from the Autopilot job

In [ ]:
# get best candidate of the job

# auto_ml_job_name = 'bank-subset-20230524-13-12'
automl = AutoML.attach(auto_ml_job_name=auto_ml_job_name)
best_candidate = automl.describe_auto_ml_job()["BestCandidate"]
best_candidate_name = best_candidate["CandidateName"]

In [79]:
best_candidate["CandidateSteps"]

[{'CandidateStepType': 'AWS::SageMaker::ProcessingJob',
  'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:827930657850:processing-job/auto-bank-all-20230626-01-43-db-1-12c3638b711d44eb9e18c95fad96c',
  'CandidateStepName': 'auto-bank-all-20230626-01-43-db-1-12c3638b711d44eb9e18c95fad96c'},
 {'CandidateStepType': 'AWS::SageMaker::TrainingJob',
  'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:827930657850:training-job/auto-bank-all-20230626-01-43-dpp7-1-3acf41de1a544b5cbdc33c2730b',
  'CandidateStepName': 'auto-bank-all-20230626-01-43-dpp7-1-3acf41de1a544b5cbdc33c2730b'},
 {'CandidateStepType': 'AWS::SageMaker::TransformJob',
  'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:827930657850:transform-job/auto-bank-all-20230626-01-43-dpp7-rpb-1-b4d1c5f005954a59b97434b',
  'CandidateStepName': 'auto-bank-all-20230626-01-43-dpp7-rpb-1-b4d1c5f005954a59b97434b'},
 {'CandidateStepType': 'AWS::SageMaker::TrainingJob',
  'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:827930657850:training-job

In [80]:
# you will need transform job for data processing/feature engineering
# and tuning job for model tuning/fitting
transform_job = ""
tuning_job = ""
for index in range(len(best_candidate["CandidateSteps"])):
    if (
        best_candidate["CandidateSteps"][index]["CandidateStepType"]
        == "AWS::SageMaker::TransformJob"
    ):
        transform_job = best_candidate["CandidateSteps"][index]["CandidateStepName"]
    if (
        best_candidate["CandidateSteps"][index]["CandidateStepType"]
        == "AWS::SageMaker::TrainingJob"
    ):
        tuning_job = best_candidate["CandidateSteps"][index]["CandidateStepName"]

print(f"transform_job:{transform_job}, tuning_job:{tuning_job}")

transform_job:auto-bank-all-20230626-01-43-dpp7-rpb-1-b4d1c5f005954a59b97434b, tuning_job:auto-bank-all-20230626-01-43GxpJ-008-c0ecb8eb


In [81]:
# If you describe the transform job, you will see the recipe of data processing
response = sagemaker_client.describe_transform_job(TransformJobName=transform_job)
response

{'TransformJobName': 'auto-bank-all-20230626-01-43-dpp7-rpb-1-b4d1c5f005954a59b97434b',
 'TransformJobArn': 'arn:aws:sagemaker:us-east-1:827930657850:transform-job/auto-bank-all-20230626-01-43-dpp7-rpb-1-b4d1c5f005954a59b97434b',
 'TransformJobStatus': 'Completed',
 'ModelName': 'auto-bank-all-20230626-01-43-dpp7-model-ad5717c1b41a4e938702e8a',
 'MaxConcurrentTransforms': 4,
 'TransformInput': {'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
    'S3Uri': 's3://sagemaker-us-east-1-827930657850/autopilot-subset-0625/full-output/auto-bank-all-20230626-01-43/preprocessed-data/tuning_data'}},
  'ContentType': 'text/csv',
  'CompressionType': 'None',
  'SplitType': 'Line'},
 'TransformOutput': {'S3OutputPath': 's3://sagemaker-us-east-1-827930657850/autopilot-subset-0625/full-output/auto-bank-all-20230626-01-43/transformed-data/dpp7/rpb',
  'Accept': 'application/x-recordio-protobuf',
  'AssembleWith': 'None',
  'KmsKeyId': ''},
 'TransformResources': {'InstanceType': 'ml.m5.4xlarge

In [82]:
# The recipe of data processing we will use in later steps
transform_model = response["ModelName"]
input_content_type = response["TransformInput"]["ContentType"]
output_accept = response["TransformOutput"]["Accept"]

In [84]:
# If you describe the training job, you will see the recipe of model tuning
response = sagemaker_client.describe_training_job(TrainingJobName=tuning_job)
# training_model = response['ModelName']
response

{'TrainingJobName': 'auto-bank-all-20230626-01-43GxpJ-008-c0ecb8eb',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:827930657850:training-job/auto-bank-all-20230626-01-43GxpJ-008-c0ecb8eb',
 'TuningJobArn': 'arn:aws:sagemaker:us-east-1:827930657850:hyper-parameter-tuning-job/auto-bank-all-20230626-01-43gxpj',
 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:827930657850:automl-job/auto-bank-all-20230626-01-43',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-827930657850/autopilot-subset-0625/full-output/auto-bank-all-20230626-01-43/tuning/auto-bank--dpp7-xgb/auto-bank-all-20230626-01-43GxpJ-008-c0ecb8eb/output/model.tar.gz'},
 'TrainingJobOutput': {'S3TrainingJobOutput': 's3://sagemaker-us-east-1-827930657850/autopilot-subset-0625/full-output/auto-bank-all-20230626-01-43/tuning/auto-bank--dpp7-xgb/auto-bank-all-20230626-01-43GxpJ-008-c0ecb8eb/output/output.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'_kfold': '5',


In [14]:
# The recipe of model tuning we will use in later steps
hyperparameters = response["HyperParameters"]
TrainingImage = response["AlgorithmSpecification"]["TrainingImage"]

## Transform the whole training dataset with the best transform model

In [17]:
timestamp_suffix = strftime("%Y%m%d-%H-%M", gmtime())
#
train_transform_job = "ap-transform-whole-train-" + timestamp_suffix
val_transform_job = "ap-transform-whole-val-" + timestamp_suffix
#'s3://sagemaker-us-east-1-827930657850/autopilot-subset-recipe-0524/output/bank-subset-20230524-13-12/preprocessed-data/tuning_data/train/chunk_0.csv'
train_transform_input = train_data_s3_path
val_transform_input = val_data_s3_path


train_transform_output = "s3://{}/{}/output/whole/train".format(bucket, prefix)
val_transform_output = "s3://{}/{}/output/whole/val".format(bucket, prefix)

In [ ]:
# kick off a new batch transform job for training and validation dataset
new_transform_job = train_transform_job
new_transform_input = train_transform_input
new_transform_output = train_transform_output

sagemaker_client.create_transform_job(
    TransformJobName=new_transform_job,
    ModelName=transform_model,
    MaxConcurrentTransforms=4,
    BatchStrategy="MultiRecord",
    TransformInput={
        "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": new_transform_input}},
        "ContentType": input_content_type,
        "SplitType": "Line",
    },
    TransformResources={"InstanceType": "ml.m5.4xlarge", "InstanceCount": 1},
    TransformOutput={"S3OutputPath": new_transform_output, "Accept": output_accept},
)

In [19]:
new_transform_job = val_transform_job
new_transform_input = val_transform_input
new_transform_output = val_transform_output

sagemaker_client.create_transform_job(
    TransformJobName=new_transform_job,
    ModelName=transform_model,
    MaxConcurrentTransforms=4,
    BatchStrategy="MultiRecord",
    TransformInput={
        "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": new_transform_input}},
        "ContentType": input_content_type,
        "SplitType": "Line",
    },
    TransformResources={"InstanceType": "ml.m5.2xlarge", "InstanceCount": 1},
    TransformOutput={"S3OutputPath": new_transform_output, "Accept": output_accept},
)

{'TransformJobArn': 'arn:aws:sagemaker:us-east-1:827930657850:transform-job/ap-transform-whole-val-20230623-13-12',
 'ResponseMetadata': {'RequestId': 'c929183d-57ee-4649-bc20-50337a640ed6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c929183d-57ee-4649-bc20-50337a640ed6',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '114',
   'date': 'Fri, 23 Jun 2023 13:13:56 GMT'},
  'RetryAttempts': 0}}

## Create a new training job
Using the hyperparameters from the best candidate and fit on a larger dataset

In [149]:
# train_data_s3_path = 's3://{}/{}/output/bank-subset-20230523-21-26/transformed-data/dpp5/rpb/train'.format(bucket, prefix)
# val_data_s3_path = 's3://{}/{}/output/bank-subset-20230523-21-26/transformed-data/dpp5/rpb/validation'.format(bucket, prefix)
train_data_path = train_transform_output
val_data_path = train_transform_output

model_s3_path = "s3://{}/{}/output/training".format(bucket, prefix)
train_instance = "ml.m5.4xlarge"
instance_count = 1
training_job_name = "ap-training-api-test-Jun29"

In [150]:
# kick off a new training job
sagemaker_client.create_training_job(
    TrainingJobName=training_job_name,
    RoleArn=role,
    AlgorithmSpecification={
        "TrainingImage": response["AlgorithmSpecification"]["TrainingImage"],
        "TrainingInputMode": "File",
    },
    HyperParameters=response["HyperParameters"],
    InputDataConfig=[
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": train_data_path,
                    "S3DataDistributionType": "ShardedByS3Key",
                }
            },
            "ContentType": output_accept,
            "CompressionType": "None",
            "RecordWrapperType": "None",
            "InputMode": "Pipe",
            # "EnableFFM": False,
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": val_data_path,
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "ContentType": output_accept,
            "CompressionType": "None",
            "RecordWrapperType": "None",
            "InputMode": "Pipe",
            # "EnableFFM": False,
        },
    ],
    OutputDataConfig={
        "KmsKeyId": "",
        "S3OutputPath": model_s3_path,
    },
    ResourceConfig={
        "InstanceType": train_instance,
        "InstanceCount": instance_count,
        "VolumeSizeInGB": 50,
    },
    StoppingCondition={"MaxRuntimeInSeconds": 86400},
)

{'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:827930657850:training-job/ap-training-api-test-Jun29',
 'ResponseMetadata': {'RequestId': '0838a3f0-444e-4ba3-90db-451b9fad84bd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0838a3f0-444e-4ba3-90db-451b9fad84bd',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '101',
   'date': 'Thu, 29 Jun 2023 16:22:46 GMT'},
  'RetryAttempts': 0}}

In [ ]:
model_name = training_job_name

# track model training progress
print("JobStatus - Secondary Status")
print("------------------------------")


info = sagemaker_client.describe_training_job(TrainingJobName=model_name)
job_run_status = info["TrainingJobStatus"]
print(info["TrainingJobStatus"] + " - " + info["SecondaryStatus"])

while job_run_status not in ("Failed", "Completed", "Stopped"):
    info = sagemaker_client.describe_training_job(TrainingJobName=model_name)
    job_run_status = info["TrainingJobStatus"]
    print(info["TrainingJobStatus"] + " - " + info["SecondaryStatus"])
    sleep(60)

## Create a model using SageMaker AutoML API & Batch Transform Predict test dataset
We will replace the original tuning model in the autopilot job with the newly fitted model

In [ ]:
# original recipe
print(best_candidate["CandidateSteps"])
# replace tuning candidate with re-fitted model
best_candidate["CandidateSteps"][3]["CandidateStepName"] = training_job_name
best_candidate["CandidateSteps"][3]["CandidateStepArn"] = info["TrainingJobArn"]
# validate
print(best_candidate["CandidateSteps"])

In [ ]:
# original recipe
print(best_candidate["InferenceContainers"][1])
# replace tuning container with re-fitted model
best_candidate["InferenceContainers"][1]["ModelDataUrl"] = info["ModelArtifacts"][
    "S3ModelArtifacts"
]
# validate
print(best_candidate["InferenceContainers"][1])

In [ ]:
# creat model by replacing recipes of best candidate
inference_response_keys = ["predicted_label", "probability"]

hack_model = automl.create_model(
    name=best_candidate_name + "-hack",
    candidate=best_candidate,
    inference_response_keys=inference_response_keys,
)

# Now you can batch transform the test dataset
batch_output = "s3://{}/{}/output/whole/batchTransformHack".format(bucket, prefix)
transformer = hack_model.transformer(
    instance_count=1, instance_type="ml.m5.xlarge", assemble_with="Line", output_path=batch_output
)

transformer.transform(
    data=test_data_s3_path, split_type="Line", content_type="text/csv", wait=False
)

In [ ]:
# model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

# primary_container = {"Image": TrainingImage, "ModelDataUrl": model_data}

# create_model_response = sagemaker_client.create_model(
#     ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
# )

# print(create_model_response["ModelArn"])

## Last Step! Evaluate the model and compare it with the baseline

In [ ]:
# make sure the baseline autopilot job is finished
print("JobStatus - Secondary Status")
print("------------------------------")


describe_response = sagemaker_client.describe_auto_ml_job(AutoMLJobName=auto_ml_baseline_job_name)
print(describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"])
job_run_status = describe_response["AutoMLJobStatus"]

while job_run_status not in ("Failed", "Completed", "Stopped"):
    describe_response = sagemaker_client.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response["AutoMLJobStatus"]

    print(
        describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"]
    )
    sleep(60)

In [ ]:
# creat model using the baseline autopilot job

automl = AutoML.attach(auto_ml_job_name=auto_ml_baseline_job_name)
best_candidate = automl.describe_auto_ml_job()["BestCandidate"]
best_candidate_name = best_candidate["CandidateName"]

baseline_model = automl.create_model(
    name=best_candidate_name,
    candidate=best_candidate,
    inference_response_keys=inference_response_keys,
)

batch_baseline_output = "s3://{}/{}/output/whole/batchTransform".format(bucket, prefix)
transformer = baseline_model.transformer(
    instance_count=1, instance_type="ml.m5.xlarge", assemble_with="Line", output_path=batch_output
)

transformer.transform(
    data=test_data_s3_path, split_type="Line", content_type="text/csv", wait=False
)

In [103]:
import re


def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

In [112]:
baseline_df = get_csv_output_from_s3(batch_baseline_output, test_file)

In [108]:
hackmodel_df = get_csv_output_from_s3(batch_output, test_file)

In [ ]:
# groundtruth
test_groundtruth = test_data["y"]

In [129]:
# compare this evaluation result with the autopilot model trained on whole dataset
from sklearn.metrics import accuracy_score

baseline_score = accuracy_score(test_groundtruth, baseline_df.iloc[:, 0])
hackmodel_score = accuracy_score(test_groundtruth, hackmodel_df.iloc[:, 0])
print(
    "Baseline model (autopilot on whole training data) score: {}, our model score: {}".format(
        baseline_score, hackmodel_score
    )
)

Baseline model (autopilot on whole training data) score: 0.8644088370963826, our model score: 0.8678077203204662


## The end